In [1]:
import pandas as pd
import json
import requests
import urllib.request
import os
from datetime import datetime
import cx_Oracle
from IPython.display import display, HTML

In [2]:
estado = 'Procesando'
ancho = 1800
display(HTML(f"<style>.container {{ width:{ancho}px !important; }}</style>"))
#df_formulas = pd.read_csv('formulas.csv',delimiter=';')

In [3]:
#Conexion a la base para levantar las formulas
#host='racexa03adm01.gobiernocba.gov.ar'
host='racexa03-scan1.gobiernocba.gov.ar'
port=1521
sid='moremigd'
user='dc'
password='dc'
sid = cx_Oracle.makedsn(host, port, service_name=sid)

connection = cx_Oracle.connect(user, password, sid, encoding="UTF-8")
cursor = connection.cursor()
cursor.execute('SELECT FOR_IMPUESTO, FOR_CPTO_ORIGEN, FOR_CPTO_CALCULO, FOR_CPTO_DISTRI, FOR_FORMULA_CALCULO, FOR_FORMULA_DISTRI, FOR_PRORRATEA, FOR_CTA_DESTINO, FOR_VIGENCIA_DESDE, FOR_VIGENCIA_HASTA, FOR_SALIDA_JSON FROM FORMULAS WHERE FOR_FECHA_BAJA IS NULL')
result_set = cursor.fetchall()
df_formulas = pd.DataFrame(result_set, columns=[col[0] for col in cursor.description])
# Close the cursor and the database connection
cursor.close()
connection.close()

In [4]:
df_formulas

,FOR_IMPUESTO,FOR_CPTO_ORIGEN,FOR_CPTO_CALCULO,FOR_CPTO_DISTRI,FOR_FORMULA_CALCULO,FOR_FORMULA_DISTRI,FOR_PRORRATEA,FOR_CTA_DESTINO,FOR_VIGENCIA_DESDE,FOR_VIGENCIA_HASTA,FOR_SALIDA_JSON
0,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,None,N,900-0000000201/03,2023-01-01,2024-12-31,N
1,INM,PRUEBA2,PRUEBA2,PRUEBA2,df.PRUEBA2,None,N,900-0000000201/03,2023-01-01,2024-12-31,N
2,AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado + df.montoRecargo,df.montoPagado,S,900-0000000201/03,2023-01-01,2024-12-31,S
3,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,df.montoPagado*.95,S,900-0000013093/02,2023-01-01,2024-12-31,S
4,AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,df.montoPagado,N,900-0000251229/08,2023-01-01,2024-12-31,S
5,None,GENERAL,GENERAL,GENERAL,df.montoPagado,df.montoPagado,N,900-0000000201/03,2023-01-01,2024-12-31,N
6,AUT,DTOPGOEL,DTOPGOEL,DTOPGOEL,df.montoPagado,df.montoPagado,N,900-0000000201/03,2023-01-01,2024-12-31,S
7,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,df.montoPagado*.05,S,900-0000251229/08,2023-01-01,2024-12-31,S
8,EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado + df.montoRecargo,df.montoPagado,N,900-0000000201/03,2023-01-01,2024-12-31,S
9,INM,FDOFIDUDES,FDOFIDUDES,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,None,N,900-0000011972/02,2023-01-01,2024-12-31,S


In [5]:
#Separo formulas de calculo y distribucion
df_formulas = df_formulas.drop(["FOR_FORMULA_DISTRI",'FOR_CTA_DESTINO'], axis=1)
#df_formulas = df_formulas_calc.drop_duplicates(subset=['FOR_IMPUESTO', 'FOR_CPTO_ORIGEN', 'FOR_CPTO_CALCULO', 'FOR_FORMULA_CALCULO', 'FOR_PRORRATEA'])
#df_formulas_distr = df_formulas.drop(["FOR_FORMULA_CALCULO"], axis=1)
#df_formulas_distr = df_formulas_distr.drop_duplicates(subset=['FOR_IMPUESTO', 'FOR_CPTO_ORIGEN', 'FOR_CPTO_CALCULO', 'FOR_CPTO_DISTRI', 'FOR_FORMULA_DISTRI', 'FOR_CTA_DESTINO', 'FOR_PRORRATEA'])
df_formulas_calc = df_formulas

In [6]:
#Abro archivo JSON
f = open('Salida.json',encoding="utf8")
#f = open('Inmo.json',encoding="utf8")
#f = open('Prueba.json',encoding="utf8")
data = json.load(f)
f.close()

In [7]:
#Se normalida para un DF
df=pd.json_normalize(data['boleta'])
datos_json = df

In [8]:
#Solo para visualizacion
datos_json = data
datos_json

{'boleta': {'idEventoPago': '000165824915',
  'idBoleta': '693416110751',
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-1234567-8',
  'apellidoNombre': 'AUTOMOTORES',
  'codBarra': '000000000000',
  'montoPagado': 7000.0,
  'idRecaudador': '934',
  'tipoPago': 'CONTADO',
  'medioPago': 'CASH',
  'cantCuotas': 0,
  'fechaVencimiento': '2023-09-12',
  'moneda': 'ARS',
  'fechaPago': '2023-09-12',
  'idCuentaContribuyente': '0001658444',
  'idReferencia': None,
  'listPagos': [{'idObligacion': '0001658434',
    'idEncomendante': 'INMO_ENCO',
    'idPago': '000165862502',
    'codBarra': '000000000000',
    'idTipoObligacion': 'INMO',
    'descripcionTipoObligacion': 'INMO',
    'fechaVencimientoObligacion': '2023-07-10',
    'fechaInicioObligacion': '2023-06-01',
    'idExterno': 'CASA',
    'montoPagado': 7000.0,
    'idPersona': '4842565083',
    'tipoDocumento': 'CUIT',
    'nroDocumento': '27-24474534-8',
    'apellidoNombre': 'MAINARDI, MARIA LORENA',


In [9]:
# Agregar la lista 'listDetallesPago' al JSON para procesar
detalles_pago = datos_json['boleta']['listPagos'][0]['listDetallesPago']
#Extraigo Lista de Pagos
lista_pagos = datos_json['boleta']['listPagos']

In [10]:
lista_pagos
id_tipo_obligacion = datos_json['boleta']['listPagos'][0]['idTipoObligacion'][:3]
id_tipo_obligacion

'INM'

In [11]:
detalle_pagos = lista_pagos
detalle_pagos

[{'idObligacion': '0001658434',
  'idEncomendante': 'INMO_ENCO',
  'idPago': '000165862502',
  'codBarra': '000000000000',
  'idTipoObligacion': 'INMO',
  'descripcionTipoObligacion': 'INMO',
  'fechaVencimientoObligacion': '2023-07-10',
  'fechaInicioObligacion': '2023-06-01',
  'idExterno': 'CASA',
  'montoPagado': 7000.0,
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-24474534-8',
  'apellidoNombre': 'MAINARDI, MARIA LORENA',
  'totalJuicio': 0.0,
  'honorariosJudiciales': 0.0,
  'gastosJudiciales': 0.0,
  'listDetallesPago': [{'tipoMovimiento': 'FDOFIDUDES',
    'montoPagado': 310175.27,
    'idTransaccion': '000160299486',
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'FDOFIDUDES'}]},
   {'tipoMovimiento': 'INMRURASMT',
    'montoPagado': 212347.5,
    'idTransaccion': '000160299486',
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'INMRURASMT'}]},
   {'tipoMovimiento': 'INMRURCOTA',
    'montoPagado': 180989.23,
    'idTransaccion': '00016

In [12]:
#Extraigo los valores de los detalles de pago
df_detalles_pago = pd.DataFrame(columns=[
    'idTransaccion',
    'tipoMovimiento',
    'montoPagado',
    # ... otros campos que quieran incluir ...
])

# Acceder a la lista de pagos
lista_pagos = datos_json['boleta']['listPagos']


for pago in lista_pagos:
    detalles_pago = pago['listDetallesPago']
    for detalle in detalles_pago:

        df_detalles_pago = pd.concat([df_detalles_pago, pd.DataFrame(detalle, index=[0])], ignore_index=True)


#df_detalles_pago


In [13]:
lista_pagos

[{'idObligacion': '0001658434',
  'idEncomendante': 'INMO_ENCO',
  'idPago': '000165862502',
  'codBarra': '000000000000',
  'idTipoObligacion': 'INMO',
  'descripcionTipoObligacion': 'INMO',
  'fechaVencimientoObligacion': '2023-07-10',
  'fechaInicioObligacion': '2023-06-01',
  'idExterno': 'CASA',
  'montoPagado': 7000.0,
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-24474534-8',
  'apellidoNombre': 'MAINARDI, MARIA LORENA',
  'totalJuicio': 0.0,
  'honorariosJudiciales': 0.0,
  'gastosJudiciales': 0.0,
  'listDetallesPago': [{'tipoMovimiento': 'FDOFIDUDES',
    'montoPagado': 310175.27,
    'idTransaccion': '000160299486',
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'FDOFIDUDES'}]},
   {'tipoMovimiento': 'INMRURASMT',
    'montoPagado': 212347.5,
    'idTransaccion': '000160299486',
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'INMRURASMT'}]},
   {'tipoMovimiento': 'INMRURCOTA',
    'montoPagado': 180989.23,
    'idTransaccion': '00016

In [14]:
detalle_pagos = df_detalles_pago
detalle_pagos

,idTransaccion,tipoMovimiento,montoPagado,caracteristica
0,000160299486,FDOFIDUDES,310175.27,"{'tipo': 'CMCCCDC', 'valor': 'FDOFIDUDES'}"
1,000160299486,INMRURASMT,212347.50,"{'tipo': 'CMCCCDC', 'valor': 'INMRURASMT'}"
2,000160299486,INMRURCOTA,180989.23,"{'tipo': 'CMCCCDC', 'valor': 'INMRURCOTA'}"
3,000160299486,INMRURAEX,105266.98,"{'tipo': 'CMCCCDC', 'valor': 'INMRURAEX'}"
4,000160299486,INMRURIPAP,32127.09,"{'tipo': 'CMCCCDC', 'valor': 'INMRURIPAP'}"
...,...,...,...,...
895,000160299486,INMRURIPAP,32127.09,"{'tipo': 'CMCCCDC', 'valor': 'INMRURIPAP'}"
896,000160299486,INMCIUCUM,91006.07,"{'tipo': 'CMCCCDC', 'valor': 'INMCIUCUM'}"
897,000160299486,DTOPGOEL,9498.94,"{'tipo': 'CMCCCDC', 'valor': 'DTOPGOEL'}"
898,000160299486,REDONDEO,9.76,"{'tipo': 'CMCCCDC', 'valor': 'REDONDEO'}"


In [15]:
df_detalles_pago['tipo_mov'] = df_detalles_pago['caracteristica'].apply(lambda x: x['valor'])
detalles_pago = df_detalles_pago.drop("caracteristica", axis=1)
detalles_pago

,idTransaccion,tipoMovimiento,montoPagado,tipo_mov
0,000160299486,FDOFIDUDES,310175.27,FDOFIDUDES
1,000160299486,INMRURASMT,212347.50,INMRURASMT
2,000160299486,INMRURCOTA,180989.23,INMRURCOTA
3,000160299486,INMRURAEX,105266.98,INMRURAEX
4,000160299486,INMRURIPAP,32127.09,INMRURIPAP
...,...,...,...,...
895,000160299486,INMRURIPAP,32127.09,INMRURIPAP
896,000160299486,INMCIUCUM,91006.07,INMCIUCUM
897,000160299486,DTOPGOEL,9498.94,DTOPGOEL
898,000160299486,REDONDEO,9.76,REDONDEO


In [16]:
df_formulas_calc = df_formulas_calc[df_formulas_calc['FOR_IMPUESTO'] == id_tipo_obligacion]
df_formulas_calc

,FOR_IMPUESTO,FOR_CPTO_ORIGEN,FOR_CPTO_CALCULO,FOR_CPTO_DISTRI,FOR_FORMULA_CALCULO,FOR_PRORRATEA,FOR_VIGENCIA_DESDE,FOR_VIGENCIA_HASTA,FOR_SALIDA_JSON
0,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N
1,INM,PRUEBA2,PRUEBA2,PRUEBA2,df.PRUEBA2,N,2023-01-01,2024-12-31,N
9,INM,FDOFIDUDES,FDOFIDUDES,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,N,2023-01-01,2024-12-31,S
10,INM,INMRURASMT,INMRURASMT,INMRURASMT,(df.INMRURASMT - df.INMCIUCUM) + ((-df.INMRUR...,N,2023-01-01,2024-12-31,S
11,INM,INMRURCOTA,INMRURCOTA,INMRURCOTA,df.INMRURCOTA,N,2023-01-01,2024-12-31,N
12,INM,INMRURAEX,INMRURAEX,INMRURAEX,df.INMRURAEX,N,2023-01-01,2024-12-31,N
13,INM,INMRURIPAP,INMRURIPAP,INMRURIPAP,df.INMRURIPAP,N,2023-01-01,2024-12-31,N
14,INM,INMFTP,INMFTP,INMFTP,df.INMFTP,N,2023-01-01,2024-12-31,N
15,INM,DTOPGOEL,DTOPGOEL,DTOPGOEL,df.DTOPGOEL,N,2023-01-01,2024-12-31,N
16,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N


In [17]:
df_final = pd.merge(df_formulas_calc, detalles_pago, how='left', left_on='FOR_CPTO_ORIGEN', right_on='tipoMovimiento').fillna({'montoPagado': 0})
df_final['tipo_mov'] = df_final['tipo_mov'].fillna(df_final['FOR_CPTO_ORIGEN'])
df_final['tipoMovimiento'] = df_final['tipoMovimiento'].fillna(df_final['FOR_CPTO_ORIGEN'])
df_final

,FOR_IMPUESTO,FOR_CPTO_ORIGEN,FOR_CPTO_CALCULO,FOR_CPTO_DISTRI,FOR_FORMULA_CALCULO,FOR_PRORRATEA,FOR_VIGENCIA_DESDE,FOR_VIGENCIA_HASTA,FOR_SALIDA_JSON,idTransaccion,tipoMovimiento,montoPagado,tipo_mov
0,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
1,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
2,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
3,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
4,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
897,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
898,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
899,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO


In [18]:
#Separo recargos de conceptos validos
df_recargo = df_final[df_final['FOR_CPTO_CALCULO'] == 'GENERAL']
df_calculo = df_final[df_final['FOR_CPTO_CALCULO'] != 'GENERAL']
df_calculo

,FOR_IMPUESTO,FOR_CPTO_ORIGEN,FOR_CPTO_CALCULO,FOR_CPTO_DISTRI,FOR_FORMULA_CALCULO,FOR_PRORRATEA,FOR_VIGENCIA_DESDE,FOR_VIGENCIA_HASTA,FOR_SALIDA_JSON,idTransaccion,tipoMovimiento,montoPagado,tipo_mov
0,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
1,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
2,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
3,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
4,INM,INMCIUCUM,INMCIUCUM,INMCIUCUM,df.INMCIUCUM,N,2023-01-01,2024-12-31,N,000160299486,INMCIUCUM,91006.07,INMCIUCUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
897,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
898,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO
899,INM,REDONDEO,REDONDEO,REDONDEO,df.REDONDEO,N,2023-01-01,2024-12-31,N,000160299486,REDONDEO,9.76,REDONDEO


In [19]:
df_calculo2 = df_calculo[['tipoMovimiento', 'FOR_FORMULA_CALCULO', 'FOR_SALIDA_JSON']]
df_calculo2

,tipoMovimiento,FOR_FORMULA_CALCULO,FOR_SALIDA_JSON
0,INMCIUCUM,df.INMCIUCUM,N
1,INMCIUCUM,df.INMCIUCUM,N
2,INMCIUCUM,df.INMCIUCUM,N
3,INMCIUCUM,df.INMCIUCUM,N
4,INMCIUCUM,df.INMCIUCUM,N
...,...,...,...
896,REDONDEO,df.REDONDEO,N
897,REDONDEO,df.REDONDEO,N
898,REDONDEO,df.REDONDEO,N
899,REDONDEO,df.REDONDEO,N


In [20]:
df_conceptos = df_calculo.groupby('tipoMovimiento')['montoPagado'].sum().reset_index()
df_conceptos

,tipoMovimiento,montoPagado
0,DTOPGOEL,949894.0
1,FDOFIDUDES,31017527.0
2,INMCIUCUM,9100607.0
3,INMFTP,1259112.0
4,INMRURAEX,10526698.0
5,INMRURASMT,21234750.0
6,INMRURCOTA,18098923.0
7,INMRURIPAP,3212709.0
8,PRUEBA2,0.0
9,REDONDEO,976.0


In [21]:
df_pivot = df_conceptos.pivot(columns='tipoMovimiento', values='montoPagado')
df_pivot = df_pivot.fillna(0)
df_pivot
total_row = df_pivot.sum(axis=0).to_frame().T
total_row

tipoMovimiento,DTOPGOEL,FDOFIDUDES,INMCIUCUM,INMFTP,INMRURAEX,INMRURASMT,INMRURCOTA,INMRURIPAP,PRUEBA2,REDONDEO
0,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0


In [22]:
df_result = pd.merge(df_calculo2, total_row, how='cross')
#df_result = df_calculo2.assign(key=1).merge(total_row.assign(key=1), on='key').drop('key', axis=1)
df = df_result
df

,tipoMovimiento,FOR_FORMULA_CALCULO,FOR_SALIDA_JSON,DTOPGOEL,FDOFIDUDES,INMCIUCUM,INMFTP,INMRURAEX,INMRURASMT,INMRURCOTA,INMRURIPAP,PRUEBA2,REDONDEO
0,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
1,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
2,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
3,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
4,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
897,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
898,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0
899,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0


In [23]:
#Aplico formula para calculo de monto final
pd.options.mode.chained_assignment = None
df_list = []
for formula in df['FOR_FORMULA_CALCULO'].unique():
#    print(formula)
    df_new = df[df['FOR_FORMULA_CALCULO'] == formula].copy()
    df_new['montoCalculado'] = pd.eval(formula)
    df_list.append(df_new)
df_calculo = pd.concat(df_list, ignore_index=True)
df_calculo

,tipoMovimiento,FOR_FORMULA_CALCULO,FOR_SALIDA_JSON,DTOPGOEL,FDOFIDUDES,INMCIUCUM,INMFTP,INMRURAEX,INMRURASMT,INMRURCOTA,INMRURIPAP,PRUEBA2,REDONDEO,montoCalculado
0,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,9100607.0
1,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,9100607.0
2,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,9100607.0
3,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,9100607.0
4,INMCIUCUM,df.INMCIUCUM,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,9100607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,976.0
897,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,976.0
898,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,976.0
899,REDONDEO,df.REDONDEO,N,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,976.0


In [24]:
v_salida = df_calculo
v_salida['idTransaccion'] = f"{datetime.now().year % 100}{datetime.now().month:02}{datetime.now().day:02}{datetime.now().hour:02}{datetime.now().minute:02}{datetime.now().second:02}{datetime.now().microsecond // 10000:02}"
v_salida['caracteristica'] = v_salida['tipoMovimiento'].apply(lambda x: f"[{{'tipo': 'CMCCCDC', 'valor':'{x}'}}]")
v_salida = v_salida[(v_salida['montoCalculado'] > 0) & (v_salida['FOR_SALIDA_JSON'] == 'S')]
v_salida

,tipoMovimiento,FOR_FORMULA_CALCULO,FOR_SALIDA_JSON,DTOPGOEL,FDOFIDUDES,INMCIUCUM,INMFTP,INMRURAEX,INMRURASMT,INMRURCOTA,INMRURIPAP,PRUEBA2,REDONDEO,montoCalculado,idTransaccion,caracteristica
101,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,1.297223e+07,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]"
102,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,1.297223e+07,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]"
103,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,1.297223e+07,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]"
104,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,1.297223e+07,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]"
105,FDOFIDUDES,df.FDOFIDUDES + ((-df.INMRURCOTA - df.INMRURAE...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,1.297223e+07,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,INMRURASMT,(df.INMRURASMT - df.INMCIUCUM) + ((-df.INMRUR...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,5.074772e+06,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]"
297,INMRURASMT,(df.INMRURASMT - df.INMCIUCUM) + ((-df.INMRUR...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,5.074772e+06,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]"
298,INMRURASMT,(df.INMRURASMT - df.INMCIUCUM) + ((-df.INMRUR...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,5.074772e+06,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]"
299,INMRURASMT,(df.INMRURASMT - df.INMCIUCUM) + ((-df.INMRUR...,S,949894.0,31017527.0,9100607.0,1259112.0,10526698.0,21234750.0,18098923.0,3212709.0,0.0,976.0,5.074772e+06,23111512170180,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]"


In [25]:
v_salida = v_salida[['tipoMovimiento','caracteristica','idTransaccion', 'montoCalculado']].round({'montoCalculado': 2})
v_salida

,tipoMovimiento,caracteristica,idTransaccion,montoCalculado
101,FDOFIDUDES,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]",23111512170180,12972228.18
102,FDOFIDUDES,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]",23111512170180,12972228.18
103,FDOFIDUDES,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]",23111512170180,12972228.18
104,FDOFIDUDES,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]",23111512170180,12972228.18
105,FDOFIDUDES,"[{'tipo': 'CMCCCDC', 'valor':'FDOFIDUDES'}]",23111512170180,12972228.18
...,...,...,...,...
296,INMRURASMT,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]",23111512170180,5074771.82
297,INMRURASMT,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]",23111512170180,5074771.82
298,INMRURASMT,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]",23111512170180,5074771.82
299,INMRURASMT,"[{'tipo': 'CMCCCDC', 'valor':'INMRURASMT'}]",23111512170180,5074771.82


In [26]:
#Elimino lista de Pagos
if 'listDetallesPago' in datos_json['boleta']['listPagos'][0]:
    del datos_json['boleta']['listPagos'][0]['listDetallesPago']
df_as_list = v_salida.to_dict(orient='records')
datos_json['boleta']['listPagos'][0]['listDetallesPago'] = df_as_list
#datos_json

In [27]:
with open("outputINMO.json", "w") as outfile:
#with open("outputEMB.json", "w") as outfile:
    json.dump(datos_json, outfile, indent=4)